In [1]:
#Philip Eisner's Sign Algorithm

#Setup
import torchvision
import torch
import numpy as np
import cv2

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

model.load_state_dict(torch.load('best_steering_model_xy.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [2]:
#Camera Creation

from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=5)

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [3]:
#Speed and turn control - For Philip's Jetbot set to 0.3 speed and 0.1 steering gain

from jetbot import Robot

robot = Robot()

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [4]:
#Robot steering sliders

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [5]:
#Color detection sliders

red_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='red')
display(red_slider)
yellow_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='yellow')
display(yellow_slider)
green_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='green')
display(green_slider)

FloatSlider(value=0.0, description='red', max=1.0)

FloatSlider(value=0.0, description='yellow', max=1.0)

FloatSlider(value=0.0, description='green', max=1.0)

In [6]:
#Main Camera update

angle = 0.0
angle_last = 0.0

import time
#Setup global variables
isGreen = False
isYellow = False
isGreen2 = False
isGreen3 = False
isYellow2 = False

greenCount = 0
greenCount2 = 0
greenCount3 = 0
yellowCount = 0
yellowCount2 = 0
def execute(change):
    #More setup
    global angle, angle_last
    global isGreen
    global isGreen2
    global isGreen3
    global isYellow
    global isYellow2
    global greenCount, yellowCount, greenCount2, greenCount3, yellowCount2
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    #red detection
    #cv2 hsv range for h is 0-180 instead of conventional 360, this caused issues and has been resolved
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) 
    #lower_red1 = np.array([0,127,114]) 
    lower_red1 = np.array([0,127,114])
    #upper_red1 = np.array([15,204,166])
    upper_red1 = np.array([8,204,166])
    #lower_red2 = np.array([345,127,114])
    lower_red2 = np.array([173,127,114]) 
    #upper_red2 = np.array([360,204,166])
    upper_red2 = np.array([180,204,166])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    frac_red = sum(sum(mask1))/np.size(mask1)
    frac_red2 = sum(sum(mask2))/np.size(mask2)
    frac_red = frac_red+frac_red2
    
    #yellow detection
    #lower_yellow1 = np.array([45, 127, 102])
    lower_yellow1 = np.array([22, 127, 102])
    #upper_yellow1 = np.array([60, 255, 153])
    upper_yellow1 = np.array([30, 255, 153])
    mask3 = cv2.inRange(hsv, lower_yellow1, upper_yellow1)
    frac_yellow = sum(sum(mask3))/np.size(mask3)
    
    #green detection
    #lower_green1 = np.array([115, 102, 102])
    lower_green1 = np.array([57, 102, 102])
    #upper_green1 = np.array([150, 255, 153])
    upper_green1 = np.array([75, 255, 153])
    mask4 = cv2.inRange(hsv, lower_green1, upper_green1)
    frac_green = sum(sum(mask4))/np.size(mask4)
    
    #update sliders
    red_slider.value = frac_red
    yellow_slider.value = frac_yellow
    green_slider.value = frac_green
    
    #Do something based off color, green and yellow are VERY hacky but I couldn't find a way to keep frames from building up during time.sleep so here we are
    #It basically checks for the color, and then will run the motors for some number of frames
    #It then will update the state booleans to match what it is doing
    #Red is just stop, green pulls over for a passenger and waits, yellow performs a U-Turn
    if (frac_red > 0.35):
        robot.stop()
    elif (frac_green > 0.20 and isGreen == False and isGreen2 == False and isGreen3 == False and isYellow == False and isYellow2 == False):
        isGreen = True
        robot.stop()
        #print("set green true")
    elif (isYellow == False and frac_yellow > 0.20 and isGreen2 == False and isGreen3 == False and isGreen == False and isYellow2 == False):
        #print("setYellow")
        isYellow = True
    elif (isGreen == False and isYellow == False and isGreen2 == False and isGreen3 == False and isYellow2 == False):
        #print("driving")
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    elif (isGreen == True):
        robot.right_motor.value = 0.4
        robot.left_motor.value = 0.1
        greenCount += 1
        #print("turning left")
        if(greenCount > 5):
            #print("finished left")
            greenCount = 0
            isGreen = False
            isGreen2 = True
            robot.stop()
    elif (isGreen2 == True):
        #print("sitting")
        robot.right_motor.value = 0
        robot.left_motor.value = 0
        greenCount2 += 1
        if(greenCount2 > 40):
            isGreen2 = False
            isGreen3 = True
            robot.stop()
    elif (isGreen3 == True):
        #print("turning right")
        robot.right_motor.value = 0.1
        robot.left_motor.value = 0.55
        greenCount += 1
        if(greenCount > 5):
            greenCount = 0
            isGreen3 = False
            robot.stop()
    elif (isYellow == True):
        robot.right_motor.value = 0.6
        robot.left_motor.value = 0.1
        #print("uturn")
        yellowCount += 1
        if(yellowCount > 10):
            yellowCount = 0
            isYellow = False
            isYellow2 = True
            robot.stop()
    elif(isYellow2 == True):
        robot.right_motor.value = 0.3
        robot.left_motor.value = 0.3
        yellowCount2 += 1
        if(yellowCount2 > 3):
            yellowCount2 = 0
            isYellow2 = False
    
execute({'new': camera.value})

In [7]:
camera.observe(execute, names='value')

In [8]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()